In [6]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [10]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
from google.colab import drive
drive.mount("/content/drive")
lending_df = pd.read_csv(Path("/content/drive/My Drive/lending_data.csv"))

# Review the DataFrame
lending_df.head()

Mounted at /content/drive


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [11]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = lending_df["loan_status"]

# Separate the X variable, the features
X = lending_df.drop("loan_status", axis=1)

In [12]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [13]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [14]:
# Check the balance of our target values
y.value_counts()

0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [15]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state =1,
                                                    stratify=y)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [16]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_logistic = LogisticRegression(solver="lbfgs", random_state=1)

# Fit the model using training data
classifier_logistic.fit(X_train, y_train)

LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [17]:
# Make a prediction using the testing data
prediction_logistic = classifier_logistic.predict(X_test)
pd.DataFrame({"Prediction": prediction_logistic, "Actual": y_test}).head()

,Prediction,Actual
36831,0,0
75818,0,1
36563,0,0
13237,0,0
43292,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [18]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, prediction_logistic)

0.9442676901753825

In [19]:
# Generate a confusion matrix for the model
confuse_matrix_log = confusion_matrix(y_test, prediction_logistic)

confuse_matrix_log

array([[18679,    80],
       [   67,   558]])

In [20]:
# Print the classification report for the model
class_report_log = classification_report(y_test, prediction_logistic)

print(class_report_log)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicted the healthy loans extraordinarily well with a precision of 100%. Furthermore, the recall and f1-scores were 100% as well which indicated it caught all of the positive cases and all the positive predictions were correct. However, the precision for the high-risk loans was only 87% correct in identifying that the model correctly predicted the instances labeled as 1. Furthermore, the recall was 89% correct in finding all positive instances. Therefore, the model could be slightly improved.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points.

In [34]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
classifer_random = RandomOverSampler(random_state=42)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = classifer_random.fit_resample(X_train, y_train)

In [35]:
# Count the distinct values of the resampled labels data
resampled_df = pd.Series(y_resampled).value_counts()
resampled_df

0    56277
1    56277
Name: loan_status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [36]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier_random_log = LogisticRegression(solver="lbfgs", random_state=1)


# Fit the model using the resampled training data
classifier_random_log.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
prediction_random_log = classifier_random_log.predict(X_test)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [37]:
# Print the balanced_accuracy score of the model
balanced_accuracy_score(y_test, prediction_random_log)

0.9959744975744975

In [38]:
# Generate a confusion matrix for the model
confuse_matrix_random_log = confusion_matrix(y_test, prediction_random_log)

confuse_matrix_random_log

array([[18668,    91],
       [    2,   623]])

In [39]:
# Print the classification report for the model
class_report_random_log = classification_report(y_test, prediction_random_log)

print(class_report_random_log)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicted the healthy loans extraordinarily well with a precision of 100%. Furthermore, the recall and f1-scores were 100% as well which indicated it caught all of the positive cases and all the positive predictions were correct. However, the precision for the high-risk loans was only 87% correct in identifying that the model correctly predicted the instances labeled as 1. However, the difference in the model comes into play here. The recall was 100% accurate which means it found all positive instances.